# 네이버영화평점

- 감성분석
- konlpy 사용
- 네이버 영화평점 데이터(https://github.com/e9t/nsmc)
- TfIdfVectorizer() 사용
- 로지스틱 회귀 사용

In [39]:
!pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [40]:
!python -m pip install --upgrade konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [41]:
# 네이버 영화 평점 데이터 다운로드
!curl https://raw.githubusercontent.com/StillWork/data/master/ratings_train.txt \
    -o ratings_train.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 14.0M  100 14.0M    0     0  37.7M      0 --:--:-- --:--:-- --:--:-- 37.7M


In [42]:
!curl https://raw.githubusercontent.com/StillWork/data/master/ratings_test.txt \
    -o ratings_test.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 4827k  100 4827k    0     0  13.8M      0 --:--:-- --:--:-- --:--:-- 13.8M


In [43]:
import konlpy
import pandas as pd
from konlpy.tag import Okt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

# 데이터 로드
df_train = pd.read_csv('ratings_train.txt', delimiter='\t', keep_default_na=False)
df_test = pd.read_csv('ratings_test.txt', delimiter='\t', keep_default_na=False)

df_train.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [55]:
df_train = df_train[:1500]
df_test = df_test[:500]

In [56]:
df_test

,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,GDNTOPCLASSINTHECLUB,0
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0
...,...,...,...
495,844432,킬링타임용 그 이상 그 이하도 아님,0
496,7296163,감정이입 되더라구요.. 이렇게 심하게 영화가 제가 된 적은 없었는데.. 저에게 해당...,1
497,10082010,군인들도 자더라ㅎㅎㅎㅎ,0
498,9611432,캡틴 듀브아 나올 때마다 너무 웃겼다. 줄리안이랑 곰의 러브스토리도 재밌었고 무엇보...,1


In [57]:
text_train = df_train['document']
y_train = df_train['label']
text_test = df_test['document']
y_test = df_test['label']

In [58]:
okt = Okt()

In [47]:
# from konlpy.tag import Kkma
# kkma=Kkma()

In [59]:
# def twitter_tokenizer(text):
#     return twitter_tag.morphs(text)

def okt_tokenizer(text):
    return okt.morphs(text)


In [60]:
cv = TfidfVectorizer(tokenizer=okt_tokenizer, min_df=3)

In [62]:
lr = LogisticRegression()

x_train = cv.fit_transform(text_train)
x_test = cv.transform(text_test)
result = lr.fit(x_train,y_train)

In [63]:
len(cv.vocabulary_)

1120

In [66]:
feature_names = cv.get_feature_names()
print(feature_names[100:200])

['같다', '같아서', '같아요', '같은', '같은데', '같음', '같이', '개', '개념', '개봉', '개연', '개인', '걍', '거', '거기', '거부', '거지', '걱정', '건', '건가', '건지', '걸', '겁나', '겁니다', '것', '게', '게임', '결국', '결말', '계속', '고', '고추', '곳곳', '공감', '공부', '공포', '공포물', '공포영화', '과', '관객', '관람', '괜찮은', '구', '구성', '구역', '국산', '군', '군대', '굿', '궁금하다', '귀신', '귀여운', '귀엽고', '그', '그것', '그냥', '그녀', '그대로', '그때', '그래도', '그래서', '그래픽', '그러나', '그러지', '그런', '그런지', '그렇게', '그렇고', '그리고', '그만', '그저', '극', '극장', '극장판', '근데', '급', '기', '기대', '기도', '기분', '기억', '기존', '긴장감', '길', '깊이', '까', '까지', '까지도', '께서', '꼭', '꽤', '끌', '끝', '끝나', '끼리', '나', '나도', '나라', '나를', '나름']


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [67]:
print("훈련 데이터 점수 : ", result.score(x_train, y_train))
print("테스트 데이터 점수 : ", result.score(x_test, y_test))

훈련 데이터 점수 :  0.8873333333333333
테스트 데이터 점수 :  0.736
